## N-Queens

**Originally Contributed by:** Matthew Helm

The N-Queens problem involves placing N queens on an N x N chessboard such that none of the queens attacks another. In chess, a queen can move vertically, horizontally, and diagonally so there cannot be more than one queen on any given row, column, or diagonal.

![](https://raw.githubusercontent.com/mthelm85/nqueens/master/n_queens4.png)

*Note that none of the queens above are able to attack any other as a result of their careful placement.*

In [10]:
using GLPK
using JuMP
using LinearAlgebra

In [11]:
# N-Queens
N = 8

8

In [12]:
model = Model(GLPK.Optimizer);

Next, let's create an N x N chessboard of binary values. 0 will represent an empty space on the board and 1 will represent a space occupied by one of our queens.

In [13]:
@variable(model, x[i=1:N, j=1:N], Bin)

8×8 Array{VariableRef,2}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]  x[1,6]  x[1,7]  x[1,8]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]  x[2,6]  x[2,7]  x[2,8]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]  x[3,6]  x[3,7]  x[3,8]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]  x[4,5]  x[4,6]  x[4,7]  x[4,8]
 x[5,1]  x[5,2]  x[5,3]  x[5,4]  x[5,5]  x[5,6]  x[5,7]  x[5,8]
 x[6,1]  x[6,2]  x[6,3]  x[6,4]  x[6,5]  x[6,6]  x[6,7]  x[6,8]
 x[7,1]  x[7,2]  x[7,3]  x[7,4]  x[7,5]  x[7,6]  x[7,7]  x[7,8]
 x[8,1]  x[8,2]  x[8,3]  x[8,4]  x[8,5]  x[8,6]  x[8,7]  x[8,8]

Now we can add our constraints:

In [14]:
# There must be exactly one queen in a given row/column
for i=1:N
    @constraint(model, sum(x[i, :]) == 1)
    @constraint(model, sum(x[:, i]) == 1)
end

In [15]:
# There can only be one queen on any given diagonal
for i in -(N-1):(N-1)
    @constraint(model, sum(diag(x,i)) <= 1)
    @constraint(model, sum(diag(reverse(x,dims=1), i)) <=1)
end

That's it! We are ready to put our model to work and see if it is able to find a feasible solution:

In [16]:
optimize!(model)

We can now review the solution that our model found:

In [17]:
solution = value.(x)

8×8 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0

![final output](https://raw.githubusercontent.com/mthelm85/nqueens/master/n_queens.png)

#### Bonus content:

Here is the code to plot your solutions with Makie.jl:

```julia
using Base.Iterators: repeated
using Makie

r = 1:N
board = isodd.(r .+ r')
scene = Scene(resolution = (250, 250))
heatmap!(scene, board, scale_plot = false, show_axis = false, colormap=:Paired)
queens = [repeated('♕', N)...]
x_positions = [findall(isequal(1.0), value.(x)[i,:])[1] for i = N:-1:1]
y_positions = 1:N
w_positions = Point2.(x_positions, y_positions)
game = scatter!(
    scene, w_positions, marker = queens,
    scale_plot = false, show_axis = false,
    markersize = 0.5, marker_offset = Vec2f0(-0.7)
)[end]
```